In [401]:
'''

2019. 11. 28.

by Yumi Kim CBBL at GNU
Screening file and Suggest Candidates which has good activity values

'''

import pandas as pd
import numpy as np
import pymysql.cursors

file_name_KCB_data = 'KCB_data_157694.csv'
data_df = pd.read_csv(file_name_KCB_data)

file_name_descriptor_list = 'qsar_descriptor_list.csv'
descriptor_list_df = pd.read_csv(file_name_descriptor_list)

file_name_all_qsar_df = 'qsar_model_list.csv'
all_qsar_df = pd.read_csv(file_name_all_qsar_df)

input_IC50_value = 0.5
screening_output_file_name = 'screening_result.csv'

def remove_specific_char(origine_srt, specific_char):
    return ''.join( c for c in origine_srt if  c not in specific_char )


def get_weight_list(weights):
    weights = remove_specific_char(weights, '[]')
    weights = weights.split(" ")

    weight_list = []
    
    for i in range(len(weights)):
        if(len(weights[i]) != 0):
            weight_list.append(float(weights[i]))
    
    return weight_list

# Get QSAR Weights and bias
bias = all_qsar_df['bias_vals'].values[0]
bias = remove_specific_char(bias, '[]')
bias = float(bias)
weights = all_qsar_df['weights_vals'].values[0]

weight_list = get_weight_list(weights)

des_list = ['smile']
des_list_except_smile = []
for i in range(descriptor_list_df.shape[0]):
    des_list.append(descriptor_list_df['0'][i])
    des_list_except_smile.append(descriptor_list_df['0'][i])
    
optimized_df = data_df[des_list]

optimized_df["Pred_IC50"] = 0
for i in range(len(weight_list)):
    optimized_df["Pred_IC50"] += optimized_df[des_list_except_smile[i]]*weight_list[i]
optimized_df["Pred_IC50"]+= bias

optimized_df = optimized_df.sort_values(["Pred_IC50"], ascending=[True])
# Selection with Condition
low_value = optimized_df['Pred_IC50'] > 0
high_value = optimized_df['Pred_IC50'] < input_IC50_value
same_bias = optimized_df['Pred_IC50'] != bias
suggest_compounds_df = optimized_df[low_value & high_value & same_bias]
suggest_compounds_df.to_csv(screening_output_file_name, index=False)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (67,121,185,186,187,188,189,190,248,249,250,251,252,253,785,786,831,850,851,1023,1024,1069,1088,1089,1140,1141,1143,1144,1145,1146,1147,1148,1149,1150,1151,1152,1182,1183,1184,1223,1224,1225,1226,1227,1228) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

In [402]:
suggest_compounds_df

,smile,AATS3m,ATSC2m,ATSC4v,C1SP3,VE3_Dt,Pred_IC50
1176,[O-][N+](=O)C2=CC=1C(=O)C(=C(NC=1C=C2)NCC(C)(C...,73.078656,508.574622,785.367375,4,-8.850157,0.091684
4781,O=C(CO/C1=N/C(=NO1)C=2C=CC=CC=2)N(CC)CCC Molecule,60.580072,276.646527,-323.977743,5,-2.252891,0.106212
1492,O=C2C(=C(NC=1C=CC(=CC=12)CCCCCC)[S](=O)C)C(=O)...,66.588609,201.593367,247.490130,2,-8.061243,0.385537
4432,O=C(CC=1C=CC=CC=1OC[C@@]2([H])ON=C(C2)C3=C/C=C...,62.678599,121.943725,-698.673742,2,-8.145496,0.437959
